<a href="https://colab.research.google.com/github/lautalom/diploUNC/blob/main/Autentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extension de longitud
El desafío consiste en crear una falsificación de un mensaje autenticado con una MAC creada a partir de una función de hash y un secreto 1.

Para obtener el mensaje a falsificar se debe hacer un requerimiento GET a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/secret-prefix-mac/<email\>/challenge
donde <email\> debe ser reemplazado por una dirección de correo electrónico registrada.

El mensaje a falsificar es una query string de la forma:

user=user@example.com&action=show&mac=55d7c19d9e4d5427d5cadd309b58a5fdfb0f78b9b40671d65c73df5ed4476784

Vemos que en este ejemplo hay tres pares clave-valor:

- user=user@example.com
- action=show
- mac=55d7c19d9e4d5427d5cadd309b58a5fdfb0f78b9b40671d65c73df5ed4476784

**La falsificación debe ser una query string válida que incluya el par `admin=true`.**

Para calcular el MAC, se construye un mensaje formado por la concatenación del resto de los pares clave-valor, ordenados alfabéticamente por clave, y eliminando los símbolos = y &. En el caso anterior, el **mensaje** sería:

<center>actionshowuseruser@example.com</center>

Sobre este mensaje se calcula un MAC de la siguiente forma:

$$MAC = \operatorname{SHA-256}(secreto || mensaje)$$

donde:
- mensaje es el mensaje conformado como se describe más arriba.
- secreto es una clave secreta de **16 bytes**.
- || denota concatenación

# Analisis
El esquema utilizado en este presenta un par de problemas.

Por un lado, distintas query strings pueden producir el mismo mensaje. Por ejemplo, el mensaje anterior también puede ser producido por la siguientes query strings:

- useru=ser@example.com&ac=tionshow
- a=ctionshowuseruser@example.com

y numerosas otras combinaciones.

En segundo lugar, SHA-256, como todos los algoritmos que usan la construcción de Merkle-Damgård, es susceptible a ataques de extensión de longitud. Como el resultado final de la función de hash puede ser tambien visto como un valor intermedio de un mensaje más largo, es posible producir una extensión del mensaje sin necesidad de conocer el secreto.

El nuevo mensaje tendrá la forma:

mensaje_extendido = mensaje || relleno || extension
donde:

- mensaje es el mensaje original.
- relleno es el relleno (padding) que debió aplicar la función de hash sobre el mensaje original, y en el cual está codificada la longitud del mensaje al cual se aplicó dicha función (lo cual incluye la longitud del secreto). Como se trata de SHA-256 la longitud está expresada como un número de 64 bits utilizando la convención big endian, ocupando los últimos 8 bytes del relleno.
- extensión es el mensaje agregado.

Para calcular el nuevo MAC, se debe utilizar una versión modificada de SHA-256, que permita declarar un IV y la cantidad de bits procesados previamente.

<center>Nuevo_MAC = SHA-256'(extension, MAC, longitud) </center>
donde

- extension es el mensaje agregado.
- MAC es el MAC original, que se usa como IV.
- longitud es la longitud del mensaje original (secreto || mensaje || relleno). Debe ser un múltiplo del tamaño de bloque de la función de hash (512 bits = 64 bytes en el caso de SHA-256).
La nueva query string debe ser tal que produzca el mensaje extendido.

Obsérvese que un par de la forma:

<center>a=ctionshowuseruser@example.com</center>

estará al principio del mensaje (porque la clave a será la primera en orden alfabético) y absorberá cualquier cosa que se le agregue hasta el próximo &.

Por lo tanto, una posible solución tendrá la forma

a=<parte restante del mensaje anterior\>&<pares que generan la extension\>&mac=<Nuevo_MAC\>
Vale la pena destacar que el relleno contendrá valores que son inválidos en una URL, por lo que habrá que utilizar un mecanismo de escape. Por ejemplo, los bytes con valor 0 deberán ser representados, en la consulta, mediante ‘%00’. No es conveniente hacer esta transformación en forma manual, sino utilizar los mecanismos de escape para URLs que proveen los distintos lenguajes de programación.

https://dl.packetstormsecurity.net/0909-advisories/flickr_api_signature_forgery.pdf

In [ ]:
import requests
import json
from base64 import b64encode, b64decode
from hashlib import sha256
from typing import Optional

In [ ]:
def find_nth_symbol_position(a, symbol, n):
  #finds index of the nth repetition of 'symbol' in string a
    count = 0
    index = -1
    while count < n:
        index = a.find(symbol, index + 1)
        if index == -1:
            break
        count += 1
    return index

def remove_symbol(a, symbol):
  while symbol in a:
        a = a.replace(symbol, '', 1)
  return a

email = "lautarolombardi19@gmail.com"
server = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/secret-prefix-mac/{email}/challenge'

resp = requests.get(server)

In [ ]:
t = resp.text
print(t)
mac = t[t.find('mac')+4:]
mac_bytes = bytes.fromhex(mac)
print(mac_bytes)
# build the string of the original message
message = t[t.find('&')+1:find_nth_symbol_position(t, '&', 2)]
message += t[:t.find('&')]
msg = remove_symbol(message, '=')
msg

user=lautarolombardi19@gmail.com&action=show&mac=059fa4afeb9b79084358dd108a734769c3740ccbd0b3eac7735269cb31d8036f
b'\x05\x9f\xa4\xaf\xeb\x9by\x08CX\xdd\x10\x8asGi\xc3t\x0c\xcb\xd0\xb3\xea\xc7sRi\xcb1\xd8\x03o'


'actionshowuserlautarolombardi19@gmail.com'

https://stackoverflow.com/questions/24183109/what-is-sha-256-padding

#Padding
append the bit '1' to the message

append k bits '0', where k is the minimum number >= 0 such that the resulting message length (modulo 512 in bits) is 448.

The message needs to be a multiple of 512 bits, this means the message length plus the number of padded bits in addition to 64 bits must equal a multiple of 512 bits. Padded bits start with an 1 and the rest are zeros.


https://blockchain-academy.hs-mittweida.de/sha-256-generator/

In [ ]:
secret_length = 16 # longitud (bytes) del secreto por consigna
# length in bits of secret||message
total_len = (len(msg) + secret_length) * 8
print(f'Byte length of msg including 16 byte secret: {total_len//8}')
# 64-bit (128 in sha512) integer that records the length of the message in big endian
len_in_padding = total_len.to_bytes(8, byteorder='big')
# amount of zeros to pad
k = (448 - (total_len + 1)) % 512
missingbits = k+1
print('Number of padded bits:', missingbits)
# Create the byte array with the initial value of 0x10
padding_bytes = bytearray([0x80])
# Calculate the number of additional zero bytes needed
zero_byte_count = (missingbits // 8 - 1)
# Add the zero bytes to the byte array
padding_bytes.extend([0x00] * zero_byte_count)
# Concatenate len_in_padding to padding_bytes
padding_bytes.extend(len_in_padding)
print('Padding length in bits =', len(padding_bytes) * 8, ' plus total len =', total_len + len(padding_bytes) * 8)
# padding_bytes


Byte length of msg including 16 byte secret: 57
Number of padded bits: 504
Padding length in bits = 568  plus total len = 1024


In [ ]:
def wrap_sha256(mac, extension, length):
    # Initialize the SHA-256 hash object with the MAC as the initial state
    sha256_hash = sha256(string=mac)

    # Update the hash with the portion of the extension starting from 'length'
    sha256_hash.update(extension)
    print(f'hashing from {extension}')

    return sha256_hash

payload = b'admintrue' + b'userlautarolombardi19@gmail.com'
mac_mio = wrap_sha256(mac_bytes, payload, 1024//8)
mac_hex = '977530d9dd4009404cc5731bc6d39ab8e81ddc70fe36680eade7a458753d7f7f' # sale del codigo del profe
print(payload, mac_mio.hexdigest())

hashing from b'admintrueuserlautarolombardi19@gmail.com'
b'admintrueuserlautarolombardi19@gmail.com' 38fb33ac8f3cb7af7b3ef1d76530cbe9627ea05c80c910e1bb3f2b097307c6c0


since server orders key,value pairs alphabetically, to replicate the prefix of 'action=show...' but also allow for the key,value pair *admin=true* we use the vulnerability that does not check keys and go 'a=' + rest of prefix so it goes first, then use padding bytes, then add the admin suffix and lastly add the 'user=username' since it wouldnt actually be a part of key,values as it was originally intended otherwise.

In [ ]:
from urllib.parse import quote
payload_str = 'a=' + msg[1:] + quote(padding_bytes) + '&admin=true&' + t[:t.find('&')]+ '&mac=' + mac_hex
payload_str

'a=ctionshowuserlautarolombardi19@gmail.com%80%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%00%01%C8&admin=true&user=lautarolombardi19@gmail.com&mac=977530d9dd4009404cc5731bc6d39ab8e81ddc70fe36680eade7a458753d7f7f'

In [ ]:
server = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/secret-prefix-mac/{email}/answer?{payload_str}'
challenge = requests.get(server)
challenge.text

'¡Ganaste!\n'

# CBC MAC

El desafío consiste en crear una falsificación de una query string autenticada con CBC-MAC.

Para obtener el mensaje a falsificar se debe hacer un requerimiento GET a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/<email\>/challenge
donde <email\> debe ser reemplazado por una dirección de correo electrónico registrada.

El mensaje a falsificar es una query string de la forma:


---
from=user@example.com&user@example.com=1000&comment=Invoice&mac=701b3768b67a68be68cee9736628cae8

---


Vemos que en este ejemplo hay cuatro pares clave-valor:

- from=user@example.com
- user@example.com=1000
- comment=Invoice
- mac=701b3768b67a68be68cee9736628cae8

La query string tiene por objeto representar una o más transferencias de dinero desde la dirección especificada como from a las direcciones especificadas. En este caso particular, se trata de una transferencia de $1000 desde user@example.com a user@example.com.

*comment* representa un comentario arbitrario, que no afecta la transacción.

Es posible agregar múltiples transferencias y comentarios, como en el siguiente ejemplo (se omite el MAC):

from=user@example.com&user@example.com=1000&comment=Invoice&user@example.edu=2500
El MAC se calcula utilizando CBC-MAC sobre toda la consulta, tal como está escrita y sin considerar el MAC:

mac = CBC-MAC("from=user@example.com&user@example.com=1000&comment=Invoice")

La falsificación debe ser una query string válida que implique la transferencia a su dirección de correo de más de $10.000.

# Analisis

Análisis
CBC-MAC no es un buen algoritmo de MAC si admitimos mensajes de longitud variable.

Sean dos mensajes $M, M'$

- $M=M_1\parallel M_2\parallel \cdots\parallel M_n$
- $M'=M'_1\parallel M_2'\parallel \cdots\parallel M_n'$

y sus respectivos tags $T, T'$

- $T = \operatorname{CBC-MAC}(M)$
- $T' = \operatorname{CBC-MAC}(M')$

Podemos generar un mensaje $M''=M\parallel (M'_1 \oplus T)\parallel M_2'\parallel \cdots\parallel  M_n'$

El Xor con $T$ anula el aporte de $M$ al tag, con lo que $\operatorname{CBC-MAC}(M'')=T'$

En este caso tenemos un solo mensaje, pero lo queremos concatener consigo mismo de manera de incrementar el monto transferido.

Es necesario tener en cuenta que debemos generar una query string válida, por lo que es probable que sea necesario escapar los caracteres que no son admisibles en una URL. Por ejemplo, los bytes con valor 0 deberán ser representados, en la consulta, mediante ‘%00’. No es conveniente hacer esta transformación en forma manual, sino utilizar los mecanismos de escape para URLs que proveen los distintos lenguajes de programación.

# Respuesta
Respuesta
La respuesta debe ser enviada con un requerimiento GET de la forma

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/<email\>/answer?<query\>
donde <email\> es una dirección de correo registrada, y <query\> es la query string falsificada, con las siguientes propiedades:

- Debe ser una query string válida, con pares atributo-valor de la forma atributo=valor separados por el carácter &.
- Debe contener un campo `from=user@example.com.
- Debe contener uno o más campos con su dirección de correo y un monto, de manera que el monto total sea mayor que $10.000.
- Puede contener uno o más campos comment.
- Debe contener un MAC correcto, que corresponda con los datos enviados.

In [ ]:
import requests
from base64 import b64encode, b64decode
from urllib.parse import quote
import hashlib

In [ ]:
email = "lautarolombardi19@gmail.com"
server = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/{email}/challenge'

first = requests.get(server)
first.text

'from=User <user@example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice&mac=a4dda59003ec18164c63663db13ce385'

In [ ]:
qs = first.text
mac_idx = qs.find('mac=')
mac_str = qs[mac_idx+4:]
mac_bytes = bytes.fromhex(mac_str)
print(mac_bytes, len(mac_bytes))

b'\xa4\xdd\xa5\x90\x03\xec\x18\x16Lcf=\xb1<\xe3\x85' 16


In [ ]:
  from urllib.parse import quote, quote_from_bytes # https://docs.python.org/3/library/urllib.parse.html#urllib.parse.quote
xornewqs = bytes([ord(a) ^ b for a, b in zip(qs, mac_bytes)])
# no funciona esto tampoco
# xornewqs = ''.join([chr(ord(a) ^ b) for a, b in zip(qs[5:], mac_bytes)])
print(xornewqs, len(xornewqs))
# prepend to previous string
payload = bytes(qs[:mac_idx - 1],'ascii')
print(payload)
# Calculate the number of bytes needed for padding
block_size = 16
padding_length = block_size - (len(payload) % block_size)
# Create the padding bytes
padding = bytes([padding_length]) * padding_length
# Combine the payload and padding
padded_payload = payload + padding + xornewqs + bytes(qs[len(xornewqs):mac_idx-1], 'ascii') + bytes(qs[mac_idx-1:], 'ascii')
padded_payload

b'\xc2\xaf\xca\xfd>\xb9ks>CZH\xc2Y\x91\xc5' 16
b'from=User <user@example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice'


b'from=User <user@example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice\x03\x03\x03\xc2\xaf\xca\xfd>\xb9ks>CZH\xc2Y\x91\xc5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice&mac=a4dda59003ec18164c63663db13ce385'

Primer test: tiene que responder que quiere mas plata

In [ ]:
answer = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/{email}/answer?'
print(quote(padded_payload, safe='&=@\ <>'))
response = requests.get(answer+quote(padded_payload, safe='&=@\ <>'))
response.text

from=User <user@example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice&mac=a4dda59003ec18164c63663db13ce385


'¡Quiero más plata!\n'

extender la logica de antes pero con mas plata, hasta cubrir la consigna de 10k al menos

In [ ]:
payload = bytes(qs[:mac_idx - 1],'ascii')
padding_length = block_size - (len(payload) % block_size)
padding = bytes([padding_length]) * padding_length
padded_payload = payload + padding
for i in range(10):
  # Combine the payload and padding
  padded_payload += xornewqs + bytes(qs[len(xornewqs):mac_idx-1], 'ascii')
  padding_length = block_size - (len(padded_payload) % block_size)
  padding = bytes([padding_length]) * padding_length
  if i==9:
    # no hay padding al ultimo
    continue
  padded_payload += padding
padded_payload += bytes(qs[mac_idx-1:], 'ascii')

In [ ]:
answer = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/{email}/answer?'
print(quote(padded_payload, safe='&=@\ <>'))
response = requests.get(answer+quote(padded_payload, safe='&=@\ <>'))
response.text

from=User <user@example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%03%03%C2%AF%CA%FD>%B9ks>CZH%C2Y%91%C5example.com>&lautarolombardi19@gmail.com=1000&comment=Invoice%03%0

'¡Ganaste!\n'